# Miniscope Quality Metrics

Visualize the miniscope quality metrics for the processed images that are stored in the DataJoint pipeline (i.e. `element-miniscope`).

If you are new to using this DataJoint pipeline for analyzing miniscope calcium imaging data, please see the [tutorial](./tutorial.ipynb) notebook for an in-depth explanation to set up and run the workflow.

This quality metrics notebook requires the data to be populated into the database using the [demo_prepare](./demo_prepare.ipynb) notebook.

In [ ]:
import os
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [ ]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
from workflow_miniscope.pipeline import miniscope

## Populate quality metrics tables

In [ ]:
miniscope.ProcessingQualityMetrics.populate(display_progress=True)

## Motion corrected summary images

In [ ]:
key = dict(
        subject="subject1",
        session_datetime=datetime.datetime(2023, 5, 11, 12, 00, 00),
        recording_id=0,
        paramset_id=0,
        curation_id=0,
    )
query = miniscope.MotionCorrection.Summary & key
query

In [ ]:
summary_images = query.fetch1()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 9))

axes[0].imshow(summary_images["average_image"][0])
axes[0].set_title('Average Image')
axes[0].set_xlabel('x (pixels)')
axes[0].set_ylabel('y (pixels)')

axes[1].imshow(summary_images["correlation_image"][0])
axes[1].set_title('Correlation Image')
axes[1].set_xlabel('x (pixels)')
axes[1].set_yticks([])
axes[1].set_yticklabels([])

axes[2].imshow(summary_images["max_proj_image"][0])
axes[2].set_title('Max projection Image')
axes[2].set_xlabel('x (pixels)')
axes[2].set_yticks([])
axes[2].set_yticklabels([])

plt.show()

## Segmentation masks

In [ ]:
mask_xpix, mask_ypix = (miniscope.Segmentation.Mask & key).fetch("mask_xpix", "mask_ypix")

mask_image = np.zeros(np.shape(summary_images["correlation_image"][0]), dtype=bool)
for xpix, ypix in zip(mask_xpix, mask_ypix):
    try:
        mask_image[ypix, xpix] = True
    except Exception as e:
        print(e)

plt.xlabel('x (pixels)')
plt.ylabel('y (pixels)')
plt.imshow(summary_images["correlation_image"][0])
plt.contour(mask_image, colors="white", linewidths=0.5)
plt.show()

## Trace quality metrics

Temporal skewness and variance of the fluorescence activity can indicate the stability of the signal over time. Changes in this metric between imaging sessions could indicate technical issues in the experimental conditions or data processing. Additionally, changes in the animal's behavior or physiological state could also affect this metric, so it is important to interpret any changes within the context of the experimental conditions and the animal's behavior and physiology. ([Stringer & Pachitariu, Current Opinion in Neurobiology 2019](https://doi.org/10.1016/j.conb.2018.11.005))

For illustrative purposes, below we will fetch and plot these metrics for a single session.

In [ ]:
key = dict(
        subject="subject1",
        session_datetime=datetime.datetime(2023, 5, 11, 12, 00, 00),
        recording_id=0,
        paramset_id=0,
        curation_id=0,
        fluorescence_channel=0
    )

query = miniscope.ProcessingQualityMetrics.Trace & key
query

In [ ]:
skewness, variance = query.fetch('skewness', 'variance')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(9, 4))

axes[0].scatter(range(len(skewness)), 
         np.sort(skewness), 
         color='black', s=0.5)
axes[0].set_title('Temporal skewness')
axes[0].set_xlabel('Cell')
axes[0].set_ylabel('Sorted skewness')

axes[1].scatter(range(len(variance)), 
         np.sort(variance), 
         color='black', s=0.5)
axes[1].set_title('Temporal variance')
axes[1].set_xlabel('Cell')
axes[1].set_ylabel('Sorted variance')

plt.show()